In [1]:
import test_config as cfg
import numpy as np

import sys, os, os.path, time, datetime
import pickle, io, json

import skimage, skimage.io, skimage.transform, skimage.filters
import sklearn, sklearn.metrics

import importlib
sys.path.append('../src/')
import modutils
import word_processing as wp
import htr_model as hm
import tqdm
import tensorflow as tf
import editdistance

In [2]:
class BaseTransformer:
    def __init__(self):
        pass
    def transform(self, x):
        return x

    
class SequentialTransformer:
    def __init__(self, *args):
        self.stages_ = args
        
    def transform(self, x):
        res = x
        for s in self.stages_:
            res = s.transform(res)
        return res
    
class LoadImageTransformer(BaseTransformer):
    def __init__(self, path):
        self.path_ = path
        
    def transform(self, x):
        if type(x) != str:
            raise Exception("LoadImageTransformer: expects filename as argument!")
        return skimage.io.imread(os.path.join(self.path_, x), as_grey=True)
    
class ConvertFloatTransformer(BaseTransformer):
    def __init__(self, min_value = 0.0, max_value = 1.0):
        self.min_ = min_value
        self.max_ = max_value
        
    def transform(self, x):
        if x.dtype in (np.float, np.float64, np.float32):
            return x
        if x.dtype == np.uint8:
            return (x / 255.0) * (self.max_ - self.min_) + self.min_
        if x.dtype == np.uint16:
            return (x / 65535.0) * (self.max_ - self.min_) + self.min_
        raise Exception("ConvertFloatTransformer: unexpected argument type!")
    
class RandomStretchTransformer(BaseTransformer):
    def __init__(self, min_scale = 0.66, max_scale = 1.5, fill_value=1.0):
        self.max_ = max_scale
        self.min_ = min_scale
        self.fill_ = fill_value
        
    def transform(self, x):
        f = np.random.uniform(self.min_, self.max_)
        return skimage.transform.rescale(x, (1.0, f), mode='constant', cval=self.fill_)
    
class TransposeTransformer(BaseTransformer):
    def __init__(self):
        pass
    def transform(self, x):
        return np.transpose(x)
    
class FitSizeTransformer(BaseTransformer):
    def __init__(self, width, height, fill_value=1.0):
        self.w_ = width
        self.h_ = height
        self.fill_ = fill_value
        self.template_ = np.ones((self.h_, self.w_)) * self.fill_
        
    def transform(self, x):
        (h, w) = x.shape
        f = max(w / self.w_, h / self.h_)
        res = self.template_.copy()
        rw = max(min(self.w_, int(w / f)), 1)
        rh = max(min(self.h_, int(h / f)), 1)
        res[0:rh, 0:rw] = skimage.transform.resize(x, (rh, rw), mode='constant', cval=self.fill_)
        return res
    
class StandardizeTransformer(BaseTransformer):
    def __init__(self):
        pass
    
    def transform(self, x):
        m = np.mean(x)
        s = np.std(x)
        if s <= 1e-9:
            return x - m
        return (x - m) / s
    
class TruncateLabelTransform(BaseTransformer):
    def __init__(self, max_cost):
        self.max_cost_ = max_cost
        
    def transform(self, x):
        if type(x) != str:
            raise Exception("TruncateLabelTransform: input expected to be of type string!")
        cost = 0
        for i in range(len(x)):
            flg = (i > 0) and (x[i] == x[i-1])
            cost += 1 + int(flg)
            if cost > max_cost:
                return x[:i]
        return x

In [3]:
with open('D:/models/htr-static-128/model_info.json', 'r') as fp:
    model_info = json.load(fp)

In [5]:
transform_pipeline = SequentialTransformer(
    FitSizeTransformer(128, 32),
    TransposeTransformer(),
    StandardizeTransformer())

In [6]:
%%time
fname = 'D:/Data/bujo_sample_v2/dataset.json'
extraction_path = os.path.join(os.path.dirname(fname),
                               os.path.basename(fname).split('.')[0])
with open(fname, 'r', encoding='utf-8') as f:
    src = json.load(f)
    
words = wp.extract_words_from_dataset(src, (1,))

transforms = [{'cutoff': 0.7, 'type': 'cutoff'},
  {'type': 'trimx'}, {'type': 'trimy'},
  {'type': 'resize', 'y': 32}, {'type': 'invert'}]

raw_lbls = [x[0] for x in words]

def load_images(path, words, transform):
    res = []
    for (i, (word, fname)) in enumerate(tqdm.tqdm(words)):
        src_image = skimage.io.imread(os.path.join(path, fname), as_grey=True)
        res_image = wp.perform_transform(src_image, transform)
        res.append(res_image)
    return res

Wall time: 62.2 ms


In [7]:
raw_imgs = load_images('D:/Data/bujo_sample_v2/dataset/', words, transforms)

  0%|                                                                                         | 0/1579 [00:00<?, ?it/s]C:\Anaconda3\lib\site-packages\skimage\io\_io.py:49: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')
C:\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
100%|█████████████████████████████████████████████████████████████████████████████| 1579/1579 [00:11<00:00, 135.78it/s]


In [9]:
tflite_fname = 'D:/models/htr-static-128/model.tflite'
interpreter = tf.lite.Interpreter(model_path=tflite_fname)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [10]:
input_details

[{'name': 'in_image',
  'index': 24,
  'shape': array([  1, 128,  32]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)}]

In [11]:
output_details

[{'name': 'out_rnn',
  'index': 25,
  'shape': array([  1,  32, 102]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0)}]

In [27]:
def run_inference(tfi, input_info, output_id, img, pipeline=BaseTransformer()):
    data = pipeline.transform(img).reshape(input_info['shape']).astype(input_info['dtype'])
    tfi.set_tensor(input_info['index'], data)
    tfi.invoke()
    return tfi.get_tensor(output_id)

In [31]:
%%time
res = []
for img in tqdm.tqdm(raw_imgs):
    res.append(run_inference(interpreter, input_details[0], output_details[0]['index'], raw_imgs[0], transform_pipeline))

100%|██████████████████████████████████████████████████████████████████████████████| 1579/1579 [00:18<00:00, 84.49it/s]


Wall time: 18.7 s


In [32]:
tflite_raw_out = np.vstack(res)

In [33]:
tflite_raw_out.shape

(1579, 32, 102)

In [20]:
with open('D:/models/htr-static-128/tf-out.pkl', 'rb') as f:
    tf_raw_out = pickle.load(f)

In [21]:
tf_raw_out.shape

(1536, 32, 102)

In [26]:
tmp.reshape(input_details[0]['shape']).astype(input_details[0]['dtype'])

array([[[0.4627424, 0.4627424, 0.4627424, ..., 0.4627424, 0.4627424,
         0.4627424],
        [0.4627424, 0.4627424, 0.4627424, ..., 0.4627424, 0.4627424,
         0.4627424],
        [0.4627424, 0.4627424, 0.4627424, ..., 0.4627424, 0.4627424,
         0.4627424],
        ...,
        [0.4627424, 0.4627424, 0.4627424, ..., 0.4627424, 0.4627424,
         0.4627424],
        [0.4627424, 0.4627424, 0.4627424, ..., 0.4627424, 0.4627424,
         0.4627424],
        [0.4627424, 0.4627424, 0.4627424, ..., 0.4627424, 0.4627424,
         0.4627424]]], dtype=float32)